# modeling6-2-nli-binaryclassifier-datapreprocessing-checkingoutput
- Based on modeling6-1-nli-binaryclassifier-datapreprocessing-checkingoutput
- preparing new inputdata 

Previously, we get the training data by 1)

    1) remove undecisive part (initial consecutive 0s)
		e.g.) 001000010101 -> 1000010101
			Not working... -> roberta-large : learn nothing, FiDT5 : good training loss, but eval loss barely changes

	2) remove undecisive part + only using positive and damaging 
		e.g.) 001000010101 -> 10, 10, 10, 1

0011101000100100011

    01* pattern -> extract 1
    *10 pattern 0> extract 0
    
    Converting each data into binary dataset2

In [14]:
import os
from util import utils
from pprint import pprint
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

## Adding ID to query for tracking

### Dev

In [2]:
path = '/data/philhoon-relevance/FiD/results/NQ_DPR/DEV/incremental_result_100'
file_name = 'ctx100.json'
file_path = os.path.join(path, file_name)

In [3]:
data = utils.open_json(file_path)

### Test

In [4]:
# path = '/data/philhoon-relevance/FiD/results/NQ_DPR/TEST/incremental_result_100'
# file_name = 'ctx100.json'
# file_path = os.path.join(path, file_name)

In [5]:
# data = utils.open_json(file_path)

### checking the result
- incremental EM is same with the previous result

In [6]:
cnt_incre = 0
cnt_all_cor = 0
cnt_all_wrong = 0
initial_zeros = 0

for instance in data:
    em_pat = instance['em_pattern']
    
    if '1' in em_pat and em_pat.startswith('0'):
        initial_zeros += 1
    if '1' in em_pat:
        cnt_incre += 1
    if em_pat.endswith('1'):
        cnt_all_cor += 1
    if em_pat.endswith('0'):
        cnt_all_wrong += 1

In [7]:
print(f'incremental top 100 em : {cnt_incre/len(data)}')
print(f'top 100 em : {cnt_all_cor/len(data)}')
print(f'initial 0s : {initial_zeros}')

incremental top 100 em : 0.6230444216055727
top 100 em : 0.5254082448327053
initial 0s : 2457


In [8]:
for id_, instance in enumerate(data):
    instance['id'] = id_

In [ ]:
# len(data)

### Dev

In [9]:
out_path = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR'
out_file_name = 'ctx100id.json'
out_file_path = os.path.join(out_path, out_file_name)

### Test

In [10]:
# out_path = '/data/philhoon-relevance/binary-classification/NQ-TEST-DPR'
# out_file_name = 'ctx100id.json'
# out_file_path = os.path.join(out_path, out_file_name)

In [11]:
# utils.save_json(data, out_file_path)

In [12]:
data_id = utils.open_json(out_file_path)

In [13]:
print(data_id[3]['id'])

3


In [14]:
len(data_id)

8757

## Split by Train and Dev
- not need for future using 1st set of K-fold

In [ ]:
random.seed(42)

In [ ]:
train, dev = train_test_split(data_id, test_size=0.2, shuffle=True)

In [ ]:
len(train)

In [ ]:
len(dev)

In [ ]:
out_path = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/train_dev'
out_file_name = 'ctx100id_train.json'
out_file_path = os.path.join(out_path, out_file_name)

In [ ]:
# utils.save_json(train, out_file_path)

In [ ]:
check = utils.open_json(out_file_path)

In [ ]:
len(check)

In [ ]:
out_path = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/train_dev'
out_file_name = 'ctx100id_dev.json'
out_file_path = os.path.join(out_path, out_file_name)

In [ ]:
# utils.save_json(dev, out_file_path)

In [ ]:
check = utils.open_json(out_file_path)

In [ ]:
len(check)

## Split by 5-fold

In [15]:
in_path = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR'
in_file_name = 'ctx100id.json'
in_file_path = os.path.join(in_path, in_file_name)
data_id = utils.open_json(in_file_path)
print(len(data_id))

8757


In [ ]:
kf = KFold(n_splits=5, random_state = 42, shuffle = True)

In [ ]:
kf.get_n_splits(data_id)

In [ ]:
output_path = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold'

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(data_id)):
    output_path_i = output_path + f'/{i+1}'
    train_file_name = f'ctx100id_split_train_{i+1}.json'
    test_file_name = f'ctx100id_split_dev_{i+1}.json'
    
    train_path = os.path.join(output_path_i, train_file_name)
    test_path = os.path.join(output_path_i, test_file_name)
    print(f'train_path : {train_path}')
    print(f'dev_path : {test_path}')
    print(f"Fold {i}:")
    print(f"  Train: index={train_index[:10]}")
    print(f"  Test:  index={test_index}")
    train_split = [data_id[j] for j in train_index]
    test_split = [data_id[j] for j in test_index]
    print(f'size of train : {len(train_split)}')
    print(f'size of dev : {len(test_split)}')
    
#     utils.save_json(train_split, train_path)
#     utils.save_json(test_split, test_path)

# Converting each data into binary dataset 
- For each split -> binary

## input 
### input file 
    : incremental inference result file
    : for need to split dataset since after transforming to binary dataset cannot track.
    
### inputformat 
    : list w/ dictionary elements
    
### keys
    'id': str : id of question for tracking
    'question': str : query 
    'answers' : list  : answers   
    'ctxs' : list w/ dictionary elements : top-100 ctx retrieved from retrieval : 
    'em_pattern': str : EM of each inference @k  
    'inference' : list : inference of each inference @k 
    'conversion' : str : existence of 10 pattern 1 if exits
    'cumulative_em' : str : existence of answer in incremental inferencing 

## Output
    - there should be option for undecisive 0s (initial consecutive zeros)
    
### output  file 
    : list w/ question and ctx pair as dictionary format 
    
### output  format 
```python
[  
   {  
       'id: str,
       'question' : str,
       'ctx' : {  
           'id' : str,  
           'text' : str,  
           'title' : str  
       }  
       'em' : str
   }  
]  
```

In [ ]:
def convert_incremental_to_binary(data, option_exclude_zeros):    
    check = 0
    output = []
    for instance in data:
        id_ = instance['id']
        query = instance['question']
        em_pattern = instance['em_pattern']
        answers = instance['answers']
        num_undecisive = len(em_pattern) - len(em_pattern.lstrip('0'))
        cumulative_em = instance['cumulative_em']
        ctxs = instance['ctxs']

        if num_undecisive > 0 and cumulative_em == '1' and option_exclude_zeros:
            check += 1
            for ctx, em in zip(ctxs[num_undecisive:], em_pattern[num_undecisive:]):
                template = {
                    'id' : id_,
                    'question' : query,
                    'ctx' : ctx,
                    'em' : em
                }
                output.append(template)
        else:
            for ctx, em in zip(ctxs, em_pattern):
                template = {
                    'id' : id_,
                    'question' : query,
                    'ctx' : ctx,
                    'em' : em
                }
                output.append(template)
    return output, check

In [ ]:
# output_wout, check_1 = convert_incremental_to_binary(test_data, option_exclude_zeros = True)

In [ ]:
# output_with, check_2 = convert_incremental_to_binary(test_data, option_exclude_zeros = False)

In [ ]:
k = 5
in_path = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold'
option_exclude_zeros = True

In [ ]:
for i in range(1, k+1):
    train_file = f'{i}/ctx100id_split_train_{i}.json'
    dev_file = f'{i}/ctx100id_split_dev_{i}.json'
    
    train_path = os.path.join(in_path, train_file)
    dev_path = os.path.join(in_path, dev_file)
    
    train_data = utils.open_json(train_path)
    dev_data = utils.open_json(dev_path)
    
    if option_exclude_zeros:
        out_path = f'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/{i}/binary_data'
        
        train_wout, _ = convert_incremental_to_binary(train_data, option_exclude_zeros = True)
        dev_wout, _ = convert_incremental_to_binary(dev_data, option_exclude_zeros = True)
        
        train_out_file = f'binary_ex_ctx100id_split_train_{i}.json'
        dev_out_file = f'binary_ex_ctx100id_split_dev_{i}.json'
        
        train_output_path = os.path.join(out_path, train_out_file)
        dev_output_path = os.path.join(out_path, dev_out_file)
        
        utils.save_json(train_wout, train_output_path)
        utils.save_json(dev_wout, dev_output_path)
        print(f'train_output_path : \n{train_output_path}')
        print(f'dev_output_path : \n{dev_output_path}')
    else:
        out_path = f'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/{i}/binary_data'
        
        train_wout, _ = convert_incremental_to_binary(train_data, option_exclude_zeros = False)
        dev_wout, _ = convert_incremental_to_binary(dev_data, option_exclude_zeros = False)
        
        train_out_file = f'binary_in_ctx100id_split_train_{i}.json'
        dev_out_file = f'binary_in_ctx100id_split_dev_{i}.json'
        
        train_output_path = os.path.join(out_path, train_out_file)
        dev_output_path = os.path.join(out_path, dev_out_file)
        
        utils.save_json(train_wout, train_output_path)
        utils.save_json(dev_wout, dev_output_path)
        print(f'train_output_path : \n{train_output_path}')
        print(f'dev_output_path : \n{dev_output_path}')
        

In [16]:
test_train = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1.json'

In [17]:
test_dev = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_dev_1.json'

In [18]:
train_1 = utils.open_json(test_train)

In [19]:
dev_1 = utils.open_json(test_dev)

In [20]:
print(len(train_1))
pprint(train_1[0])

681487
{'ctx': {'id': '533920',
         'text': 'he was unsatisfied with the book. Ellison ultimately wrote '
                 'more than 2,000 pages of this second novel but never '
                 'finished it. Ellison died on April 16, 1994 of pancreatic '
                 'cancer and was interred in a crypt at Trinity Church '
                 'Cemetery in the Washington Heights neighborhood of Upper '
                 'Manhattan. He was survived by his second wife, Fanny Ellison '
                 '(November 27, 1911 – November 19, 2005). "Invisible Man" won '
                 'the 1953 US National Book Award for Fiction. The award was '
                 'his ticket into the American literary establishment. He '
                 'eventually was admitted to the American Academy of Arts and '
                 "Letters, received two President's",
         'title': 'Ralph Ellison'},
 'em': '0',
 'id': 1,
 'question': 'how many pages is invisible man by ralph ellison'}


In [21]:
print(len(dev_1))
pprint(dev_1[0])

169978
{'ctx': {'id': '11828866',
         'text': 'Does He Love You "Does He Love You" is a song written by '
                 'Sandy Knox and Billy Stritch, and recorded as a duet by '
                 'American country music artists Reba McEntire and Linda '
                 'Davis. It was released in August 1993 as the first single '
                 'from Reba\'s album "Greatest Hits Volume Two". It is one of '
                 'country music\'s several songs about a love triangle. "Does '
                 'He Love You" was written in 1982 by Billy Stritch. He '
                 'recorded it with a trio in which he performed at the time, '
                 'because he wanted a song that could be sung by the other two '
                 'members',
         'title': 'Does He Love You'},
 'em': '1',
 'id': 0,
 'question': 'who sings does he love me with reba'}


In [25]:
train_id_lst = []
for instance in train_1:
    train_id_lst.append(instance['id'])
print(len(list(set(train_id_lst))))

7005


In [36]:
check_dict = {
    '1' : 0,
    '0' : 0
}
for instance in train_1:
    if instance['em'] == '1':
        check_dict['1'] += 1
    if instance['em'] == '0':
        check_dict['0'] += 1

In [37]:
check_dict

{'1': 358747, '0': 322740}

In [41]:
check_dict['0'] + check_dict['1']

681487

In [42]:
print(check_dict['1']/(check_dict['0'] + check_dict['1']))
print(check_dict['0']/(check_dict['0'] + check_dict['1']))

0.5264179654197365
0.47358203458026343


In [33]:
check_dict

{'1': 1, '0': 0}

In [40]:
len(train_1)

681487

In [23]:
dev_id_lst = []
for instance in dev_1:
    dev_id_lst.append(instance['id'])
print(len(list(set(dev_id_lst))))

1752


# Converting Test data into binary dataset
- Here include zero

In [ ]:
input_path = '/data/philhoon-relevance/binary-classification/NQ-TEST-DPR'
input_file = 'ctx100id.json'
output_file = 'binary_in_ctx100id_test.json'

In [ ]:
input_file_path = os.path.join(input_path, input_file)
data_in = utils.open_json(input_file_path)

In [ ]:
test_output_path = os.path.join(input_path, output_file)
print(test_output_path)

In [ ]:
test_with, _ = convert_incremental_to_binary(data_in, option_exclude_zeros = False)
utils.save_json(test_with, test_output_path)

In [ ]:
check_test = utils.open_json(test_output_path)

In [ ]:
print(len(check_test))
pprint(check_test[0])


In [ ]:
test_id_lst = []
for instance in check_test:
    test_id_lst.append(instance['id'])
print(len(list(set(test_id_lst))))


# Converting each data into binary dataset2
- For each split -> remove undecisive part + only using positive and damaging 

0011101000100100011

    01* pattern -> extract 1
    *10 pattern 0> extract 0
    
    Converting each data into binary dataset2
    
0000000000000000000

    pass

## input 
### input file 
    : incremental inference result file
    : for need to split dataset since after transforming to binary dataset cannot track.
    
### inputformat 
    : list w/ dictionary elements
    
### keys
    'id': str : id of question for tracking
    'question': str : query 
    'answers' : list  : answers   
    'ctxs' : list w/ dictionary elements : top-100 ctx retrieved from retrieval : 
    'em_pattern': str : EM of each inference @k  
    'inference' : list : inference of each inference @k 
    'conversion' : str : existence of 10 pattern 1 if exits
    'cumulative_em' : str : existence of answer in incremental inferencing 

## Output
    - there should be option for undecisive 0s (initial consecutive zeros)
    
### output  file 
    : list w/ question and ctx pair as dictionary format 
    
### output  format 
```python
[  
   {  
       'id: str,
       'question' : str,
       'ctx' : {  
           'id' : str,  
           'text' : str,  
           'title' : str  
       }  
       'inference' : str,
       'em' : str
   }  
]  
```

In [47]:
import os
import re

In [16]:
# i = 1

In [17]:
# input_path = f'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold'

In [18]:
# train_file = f'{i}/ctx100id_split_train_{i}.json'
# dev_file = f'{i}/ctx100id_split_dev_{i}.json'

In [19]:
# input_train_file = os.path.join(input_path, train_file)
# input_dev_file = os.path.join(input_path, train_file)

In [20]:
# input_train = utils.open_json(input_train_file)

In [21]:
# input_dev = utils.open_json(input_dev_file)

In [22]:
# len(input_train)

7005

In [298]:
# len(input_dev)

1752

In [27]:
# print(input_train[0].keys())

dict_keys(['question', 'answers', 'ctxs', 'em_pattern', 'inference', 'conversion', 'cumulative_em', 'id'])


In [299]:
# instance = input_train[3]

In [303]:
# id_ = instance['id']
# query = instance['question']
# em_pattern = instance['em_pattern']
# answers = instance['answers']
# num_undecisive = len(em_pattern) - len(em_pattern.lstrip('0'))
# cumulative_em = instance['cumulative_em']
# ctxs = instance['ctxs']

In [304]:
# em_pattern

In [305]:
# num_undecisive

In [306]:
def get_definite_pos_neg(test_em):
    positive_pos = []
    if test_em.startswith('1'):
        positive_pos.append(0)
    iter_ = re.finditer(r'01', test_em)
    for m in iter_:
        pos_ = m.start() + 1
        positive_pos.append(pos_)

    negative_pos = []
    iter_ = re.finditer(r'10', test_em)
    for m in iter_:
        pos_ = m.start() + 1
        negative_pos.append(pos_)
    
    return positive_pos, negative_pos

In [307]:
def convert_incremental_to_decisive_binary(data):
    output = []
    
    for instance in data:
        id_ = instance['id']
        query = instance['question']
        em_pattern = instance['em_pattern']
        answers = instance['answers']
        num_undecisive = len(em_pattern) - len(em_pattern.lstrip('0'))
        cumulative_em = instance['cumulative_em']
        ctxs = instance['ctxs']
        
        if num_undecisive == 100:
            continue
        else:
            pos_indices, neg_indices = get_definite_pos_neg(em_pattern)

            if pos_indices:
                for pos_ in pos_indices:
                    if em_pattern[pos_] == '1':
                        ctx_ = ctxs[pos_]
                        template = {
                            'id' : id_,
                            'question' : query,
                            'ctx' : ctx_,
                            'em' : em_pattern[pos_]
                        }
                        output.append(template)
                    else:
                        print('wrong')
                        print(f'id_ : {id_}')
                        break

            if neg_indices:                
                for neg_ in neg_indices:
                    if em_pattern[neg_] == '0':
                        ctx_ = ctxs[neg_]
                        template = {
                            'id' : id_,
                            'question' : query,
                            'ctx' : ctx_,
                            'em' : em_pattern[neg_]
                        }
                        output.append(template)
                    else:
                        print('wrong')
                        print(f'id_ : {id_}')
                        break

    return output


In [209]:
input_path = f'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold'

In [210]:
train_file = f'{i}/ctx100id_split_train_{i}.json'
dev_file = f'{i}/ctx100id_split_dev_{i}.json'

In [211]:
input_train_file = os.path.join(input_path, train_file)
input_dev_file = os.path.join(input_path, dev_file)

In [212]:
input_train = utils.open_json(input_train_file)

In [213]:
input_dev = utils.open_json(input_dev_file)

In [214]:
len(input_train)

7005

In [215]:
len(input_dev)

1752

In [216]:
train_decisive_binary = convert_incremental_to_decisive_binary(input_train)

In [217]:
len(train_decisive_binary)

6721

In [296]:
cnt_0 = 0
cnt_1 = 0

for ins in train_decisive_binary:
    if ins['em'] == '0':
        cnt_0 += 1
    if ins['em'] == '1':
        cnt_1 += 1 

In [297]:
print(cnt_0)
print(cnt_1)

1516
5205


## Converting each 5-fold

In [311]:
k = 5
in_path = f'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold'

In [315]:
for i in range(1, k+1):
    
    # input_file
    train_file = f'{i}/ctx100id_split_train_{i}.json'
    dev_file = f'{i}/ctx100id_split_dev_{i}.json'
    
    train_path = os.path.join(in_path, train_file)
    dev_path = os.path.join(in_path, dev_file)
    
    print(f'train_path : {train_path}')
    print(f'dev_path : {dev_path}')
    train_data = utils.open_json(train_path)
    dev_data = utils.open_json(dev_path)
    
    
    out_path = f'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/{i}/decisive_binary_data'
    
    os.makedirs(out_path, exist_ok = True)
    
    train_out = convert_incremental_to_decisive_binary(train_data)
    dev_out = convert_incremental_to_decisive_binary(dev_data)
    
    print(f'len(train_out) : {len(train_out)}')
    print(f'len(dev_out) : {len(dev_out)}')
    
    train_out_file = f'binary_decisive_ctx100id_split_train_{i}.json'
    dev_out_file = f'binary_decisive_ctx100id_split_dev_{i}.json'

    train_output_path = os.path.join(out_path, train_out_file)
    dev_output_path = os.path.join(out_path, dev_out_file)
    
    utils.save_json(train_out, train_output_path)
    utils.save_json(dev_out, dev_output_path)
    print(f'train_output_path : \n{train_output_path}')
    print(f'dev_output_path : \n{dev_output_path}')
    print('=====')
        

train_path : /data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/ctx100id_split_train_1.json
dev_path : /data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/ctx100id_split_dev_1.json
len(train_out) : 6721
len(dev_out) : 1704
train_output_path : 
/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/decisive_binary_data/binary_decisive_ctx100id_split_train_1.json
dev_output_path : 
/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/decisive_binary_data/binary_decisive_ctx100id_split_dev_1.json
=====
train_path : /data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/2/ctx100id_split_train_2.json
dev_path : /data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/2/ctx100id_split_dev_2.json
len(train_out) : 6736
len(dev_out) : 1689
train_output_path : 
/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/2/decisive_binary_data/binary_decisive_ctx100id_split_train_2.json
dev_output_path : 
/data/philho

In [316]:
for i in range(1, k+1):
    check_path = f'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/{i}/decisive_binary_data'
    train_out_file = f'binary_decisive_ctx100id_split_train_{i}.json'
    dev_out_file = f'binary_decisive_ctx100id_split_dev_{i}.json'
    
    train_file = os.path.join(check_path, train_out_file)
    dev_file = os.path.join(check_path, dev_out_file)
    
    train_data = utils.open_json(train_file)
    dev_data = utils.open_json(dev_file)
    
    print(f'train_data : {len(train_data)}')
    print(f'dev_data : {len(dev_data)}')

train_data : 6721
dev_data : 1704
train_data : 6736
dev_data : 1689
train_data : 6764
dev_data : 1661
train_data : 6667
dev_data : 1758
train_data : 6812
dev_data : 1613


In [295]:
train_decisive_binary[14]

{'id': 25,
 'question': 'when is star vs the forces of evil coming back on 2017',
 'ctx': {'id': '17192444',
  'title': 'Star vs. the Forces of Evil',
  'text': '"The Battle for Mewni" and consisted of the first four episodes. A live chat featuring Star and Marco was aired on Disney XD on July 17. The remaining third-season episodes started airing on November 6, 2017. A fourth season was also ordered ahead of the third season premiere. "Star vs. the Forces of Evil" premiered in Canada on the DHX-owned Disney XD on April 6, 2015, and was later moved to the Corus-owned Disney XD on December 1. The series premiered on Disney XD channels in the United Kingdom and Ireland on April 16, 2015, in Australia on August 3,'},
 'em': '1'}

In [282]:
i = 9

In [283]:
input_train[i]['id']

11

In [284]:
input_train[i]['em_pattern']

'1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111'

In [285]:
pos = len('1') - 1

In [286]:
input_train[i]['ctxs'][pos]

{'id': '2279250',
 'title': 'Robinson projection',
 'text': 'purpose world maps in 1988, replacing the Van der Grinten projection. In 1998 NGS abandoned the Robinson projection for that use in favor of the Winkel tripel projection, as the latter "reduces the distortion of land masses as they near the poles". The Robinson projection is neither equal-area nor conformal, abandoning both for a compromise. The creator felt this produced a better overall view than could be achieved by adhering to either. The meridians curve gently, avoiding extremes, but thereby stretch the poles into long lines instead of leaving them as points. Hence, distortion close to the poles is severe,'}

In [189]:
# test_em = '0011101000100100011'
# test_em = '0000000000000000001'
# test_em = '0000111000011110001'
# test_em = '1000111000011110000'
test_em = '1111111111111111111'

In [198]:
test_em[0]

'1'

In [190]:
pos_indices, neg_indices = get_definite_pos_neg(test_em)
print(f'pos_indices : {pos_indices}')
print(f'neg_indices : {neg_indices}')

pos_indices : [0]
neg_indices : []


In [191]:
for p in pos_indices:
    print(test_em[p])

1


In [192]:
for n in neg_indices:
    print(test_em[n])

In [193]:
not pos_indices

False

In [197]:
not neg_indices

True

In [ ]:
def convert_incremental_to_binary(data, option_exclude_zeros):    
    check = 0
    output = []
    for instance in data:
        id_ = instance['id']
        query = instance['question']
        em_pattern = instance['em_pattern']
        answers = instance['answers']
        num_undecisive = len(em_pattern) - len(em_pattern.lstrip('0'))
        cumulative_em = instance['cumulative_em']
        ctxs = instance['ctxs']

        if num_undecisive > 0 and cumulative_em == '1' and option_exclude_zeros:
            check += 1
            for ctx, em in zip(ctxs[num_undecisive:], em_pattern[num_undecisive:]):
                template = {
                    'id' : id_,
                    'question' : query,
                    'ctx' : ctx,
                    'em' : em
                }
                output.append(template)
        else:
            for ctx, em in zip(ctxs, em_pattern):
                template = {
                    'id' : id_,
                    'question' : query,
                    'ctx' : ctx,
                    'em' : em
                }
                output.append(template)
    return output, check

In [1]:
out_path = f'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/{i}/decisive_binary_data'

NameError: name 'i' is not defined